<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Total Knee Replacement: Path Analysis
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" styTotal Knee Replacement: Path Analysisle="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Pathing is the process of discovering a sequence of antecedent actions that occur prior to a specific event of interest on sessionized data. Pathing discovers the most salient patterns across a group of individuals or entities based on which further actions are considered. Pathing allows you to provide an explanation of the relation and the relative importance of each factor.</p>

<p style = 'font-size:16px;font-family:Arial'>The <code>nPath</code> function provides a flexible pattern-matching capability that lets you specify complex patterns in the input data and define the values that are output for each matched input set.</p>


<p style = 'font-size:18px;font-family:Arial'><b>Business Value</b></p>
<li style = 'font-size:16px;font-family:Arial'>Comprehensive health predictions and a reduced number of false positive and false negative results.</li>
<li style = 'font-size:16px;font-family:Arial'>Reduced cost to patients and hospitals.</li>
<li style = 'font-size:16px;font-family:Arial'>Identify patterns and symptoms leading to total knee replacements.</li>
<li style = 'font-size:16px;font-family:Arial'>Provides an understanding of various touchpoints that lead to total Knee Replacement.</li></p>

<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>

<p style = 'font-size:16px;font-family:Arial'>With the advantages of Vantage and ClearScape Analytics, we bring key differentiating capabilities that set us apart from the current fragmented state many of our customers and prospects face. Analyzing behavior across multiple touchpoints poses many challenges.  Data is disconnected, or when it is available, typically exists in vast quantities that make timely, accurate analysis impossible.  If this analysis is available, it is typically only available inside of a closed, proprietary ecosystem.</p>
<p style = 'font-size:16px;font-family:Arial'>Fortunately, Vantage allows users to unlock the value in their various journeys by</p>
<li style = 'font-size:16px;font-family:Arial'>Accessing data across multiple, disconnected silos, and</li>
<li style = 'font-size:16px;font-family:Arial'>Applying unique, massively-scalable analytics capabilities which allows organizations to understand a complete and accurate picture of the user journey.</li></p>
<p style = 'font-size:16px;font-family:Arial'>With Vantage, these analytics can be run in real-time for any data volume or number of users – providing an accurate, timely view of overall user experience. And of course, Vantage and ClearScape Analytics allow for at-scale processing across all the digital interactions that make up these journeys.</p>
<p style = 'font-size:16px;font-family:Arial'>Furthermore, this analysis can be passed to additional ML and AI capabilities available in ClearScape Analytics, helping to develop more accurate and actionable predictive models.</p>


<p style = 'font-size:16px;font-family:Arial'>The goal is to prepare data using Vantage in-database analytic functions, then to visualize them in Sankey charts in order to view care pathways leading to a total knee replacement. </p>

<p style = 'font-size:16px;font-family:Arial'>The data used in this demonstration represents procedures for injuries to the knee.</p>
<li style = 'font-size:16px;font-family:Arial'>289,839 events for various members</li>
<li style = 'font-size:16px;font-family:Arial'>51 states/territories</li>
<li style = 'font-size:16px;font-family:Arial'>13 procedures</li>
<li style = 'font-size:16px;font-family:Arial'>4 months</li>

</p>
<p style = 'font-size:16px;font-family:Arial'>In this demo, we will leverage teradataml virtual dataframes with Vantage Analytic in-database functions - Sessionize() and nPath() - to create a Sankey visualization using open source package, Plotly.</p>

</header>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage.</b></p>


<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
#%%capture 
#!pip install colorlover
#!pip install teradataml --upgrade teradataml

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
import json
import getpass
import pandas as pd

from teradataml.dataframe.dataframe import DataFrame
from teradataml import *

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import defaultdict
import plotly.offline as offline
import colorlover as cl
offline.init_notebook_mode()
display.max_rows=5

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Knee_Replacement_nPath_Python.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>


In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_KneeReplacement_cloud');"
 # Takes about 10 seconds
%run -i ../run_procedure.py "call get_data('DEMO_KneeReplacement_local');"
 # Takes about 30 seconds

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start by creating a "Virtual DataFrame" that points directly to the dataset in Vantage. We then begin our analysis by checking the shape of the DataFrame and examining the data types of all its columns.</p>




In [ ]:
knee_claims = DataFrame(in_schema('DEMO_KneeReplacement', 'Knee_Replacement'))
knee_claims

<p style = 'font-size:16px;font-family:Arial'>There are 10 columns mentioning the procedure code , diagnosis code and shortdesc which mentions the procedure to be conducted and the amount required for the procedure along with other personal details.</p>


<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. Sessionize() Function</b></p>


<p style = 'font-size:16px;font-family:Arial'>The Sessionize() Function in the Advanced SQL Engine (SQLE) was originally designed to capture a sequence of web clicks by one user that are separated by at most *n* seconds. However, we've also found it useful to define a sequence of events along a longitudinal record of a member's patient journey. In this case, we will define a session as one day.</p>

<p style = 'font-size:16px;font-family:Arial'>one day = 60 seconds x 60 minutes x 24 hours = 86,400 seconds</p>

<p style = 'font-size:16px;font-family:Arial'>Here we rename the column shortdesc to procedure and then use Sessionize function on that data. </p>

In [ ]:
knee_claims = knee_claims.assign(proceduredesc=knee_claims.shortdesc)
knee_claims = knee_claims.drop('shortdesc', axis=1)
knee_claims.columns

In [ ]:
knee_claims_sessions = Sessionize(data = knee_claims,
                               data_partition_column = ["memberid"],
                               data_order_column = ["tstamp"],
                               time_column = "tstamp",
                               time_out = 86400.0, # must be passed as a float
                               )


<p style = 'font-size:16px;font-family:Arial'>Note the SESSIONID column that is appended to the end.</p>

In [ ]:
knee_claims_sessions.result.sort(['memberid', 'tstamp'])

<p style = 'font-size:16px;font-family:Arial'>The result is ordered sessions per member ID.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>5. nPath() Function </b></p>

<p style = 'font-size:16px;font-family:Arial'>More insight is gained when combining the events into a path, then further analyzing a population of paths - that is, more common paths, paths with desirable or undesirable outcomes.</p> 

<p style = 'font-size:16px;font-family:Arial'>Now let us build a path on the sessionization results.</p>

In [ ]:
from teradataml import ConvertTo
knee_claims_sessions_out = ConvertTo(data = knee_claims_sessions.result,
                           target_columns = ['memberid','proccode','diagcode','amount', 'tstamp','firstname','lastname','email', 'state','proceduredesc','SESSIONID'],
                           target_datatype = ["integer","integer","VARCHAR(charlen=20,charset=LATIN,casespecific=NO)", "integer","timestamp","VARCHAR(charlen=50,charset=LATIN,casespecific=NO)",
                                              "VARCHAR(charlen=50,charset=LATIN,casespecific=NO)","VARCHAR(charlen=255,charset=LATIN,casespecific=NO)","VARCHAR(charlen=5,charset=LATIN,casespecific=NO)",
                                              "VARCHAR(charlen=500,charset=LATIN,casespecific=NO)","integer"])
knee_claims_sessions_out.result

<p style = 'font-size:16px;font-family:Arial'><b>Create paths ending in Total Knee Replacement</b> </p>

In [ ]:
npath_sessions = NPath(data1 = knee_claims_sessions_out.result, 
                      data1_partition_column = ['memberid'], 
                      data1_order_column = ['tstamp','proceduredesc'], 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['proceduredesc in (\'TOTAL KNEE REPLACEMENT\') as E', 'proceduredesc not in (\'TOTAL KNEE REPLACEMENT\') as A'], 
                      pattern = 'A{1,5}*.E', 
                      result = ['ACCUMULATE(proceduredesc OF ANY(A,E)) AS path',
                            'FIRST(diagcode OF ANY(A,E)) AS diagnosis', 
                            'FIRST(memberid OF ANY(A,E)) AS unique_memberid',   
                            'COUNT(proceduredesc OF ANY(A,E)) AS proc_cnt',
                            'FIRST(tstamp OF ANY(E)) AS starttime'])

npath_sessions.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(5)

In [ ]:
knee_path_results = npath_sessions.result

<p style = 'font-size:16px;font-family:Arial'><b>Create paths starting with Range of Motion Testing, has Physical Therapy NEC in the middle, and ends in Total Knee Replacement</b></p>

In [ ]:
knee_path_results.select(['path','unique_memberid']).groupby('path').count().sort('count_unique_memberid', ascending=False)

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>6. Sankey Charts</b></p>
<p style = 'font-size:16px;font-family:Arial'>Sankey charts can help visualize pathways and volume of events on the  most common paths.  Teradata Vantage Cloud does have built-in visualizations, but if users don't have access to these capabilities, or perhaps there is a preference to work in Python, the Plotly package provides Sankey charting capabilities.  Below the Python function reads output from nPath as pandas dataframe and plots Sankey chart. This method will be used throughout for plotting Sankey.</p>

<p style = 'font-size:14px;font-family:Arial'><i><b>**Note: The code in the below cell is just the definition of the function created for Sankey charts, so will not show any chart. The function is used later in the notebook below wherever charts are needed </b></i></p>

In [ ]:
#Convert Teradata nPath output to plotly Sankey
#can handle paths up to 999 links in length
import plotly.graph_objects as go
def sankeyPlot(res, title_text="Basic Sankey Path"):
    npath_pandas = res
    
    dataDict = defaultdict(int)
    eventDict = defaultdict(int)
    maxPath = npath_pandas['count_unique_memberid'].max()
    
    for index, row in npath_pandas.iterrows():
        rowList = row['path'].replace('[','').replace(']','').split(',')
        pathCnt = row['count_unique_memberid']
        pathLen = len(rowList)
        for i in range(len(rowList)-1):
            leftValue = str(150 + i + maxPath - pathLen) + rowList[i].strip()
            rightValue = str(150 + i + 1 + maxPath - pathLen) + rowList[i+1].strip()
            valuePair = leftValue + '+' + rightValue
            dataDict[valuePair] += pathCnt
            eventDict[leftValue] += 1
            eventDict[rightValue] += 1
    
    eventList = []
    for key,val in eventDict.items():
        eventList.append(key)
    
    sortedEventList = sorted(eventList)
    sankeyLabel = []
    for event in sortedEventList:
        sankeyLabel.append(event[3:])
    
    sankeySource = []
    sankeyTarget = []
    sankeyValue = []

    for key,val in dataDict.items():
        sankeySource.append(sortedEventList.index(key.split('+')[0]))
        sankeyTarget.append(sortedEventList.index(key.split('+')[1]))
        sankeyValue.append(val)
    
    sankeyColor = []
    for i in sankeyLabel:
        sankeyColor.append('blue')
    
    sankeyChart = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = 'black',
            width = 0.5
          ),
          label = sankeyLabel,
          color = sankeyColor
        ),
        link = dict(
            source = sankeySource,
            target = sankeyTarget,
            value = sankeyValue
        )
      )
    layout =  dict(
        title = title_text,
        font = dict(
          size = 10
        )
    )

    link = dict(source = sankeySource, target = sankeyTarget, value = sankeyValue, color='white')
    node = dict(label=sankeyLabel, pad=15, thickness=10, color='orange')
    data=go.Sankey(link=link, node=node)
    # plot 
    fig=go.Figure(data)
    fig.update_layout(
            hovermode ='closest',
            title = title_text,
            title_font_size=20,
            font = dict(size = 10, color = 'white'),
            plot_bgcolor='black',
            paper_bgcolor="#585958"
            )
    
    fig.show()  

In [ ]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
res = knee_path_results\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(10)\
                    .to_pandas()
sankeyPlot(res,"Path to Total Knee Replacement")

<p style = 'font-size:16px;font-family:Arial'>The TOP 3 events that led to <b>Total Knee Replacement</b> are <b>Knee Arthroscopy</b>, <b>Knee Joint Biopsy</b> and <b>Other Diagnostic Physical Therapy Procedure.</p>

<p style = 'font-size:16px;font-family:Arial'>To check the details of any path or node we can move the mouse pointer over it and check details. For example if you move the pointer over the path having the largest width at the right most path going towards the right most node(Total Knee Replacement) it shows <b>6, source: Knee Arthroscopy, target: Total Knee Replacement.</b> It means there were 6 cases where Knee Arthroscopy led to recommendation of Total Knee Replacement.</p>

<p style = 'font-size:16px;font-family:Arial'>When the pointer is moved over a Node, for example when the pointer is on the long Node at the right in the middle <b>OTH Arthrotomy-Knee </b>  it shows <b>incoming flow count: 2 and outgoing flow count: 3</b> which means that there are 2 different events which lead to OTH Arthrotomy-Knee after which the next 3 events are Knee Arthoroscopy, Knee Joint Biopsy and Other Diagnostic Physical. Similarly other nodes and paths can be analyzed.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Check paths that led to Knee Arthroscopy</b></p>
<p style = 'font-size:16px;font-family:Arial'>As seen in the above analysis ,Knee replacement was suggested most number of times after knee arthroscopy, so let's try to analyze what events led to Knee Arthroscopy</p>

In [ ]:
npath_sessions1 = NPath(data1 = knee_claims_sessions_out.result, 
                      data1_partition_column = ['memberid'], 
                      data1_order_column = ['tstamp','proceduredesc'], 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['proceduredesc in (\'KNEE ARTHROSCOPY\') as E', 'proceduredesc not in (\'KNEE ARTHROSCOPY\') as A'], 
                      pattern = 'A{1,5}*.E', 
                      result = ['ACCUMULATE(proceduredesc OF ANY(A,E)) AS path',
                            'FIRST(diagcode OF ANY(A,E)) AS diagnosis', 
                            'FIRST(memberid OF ANY(A,E)) AS unique_memberid',   
                            'COUNT(proceduredesc OF ANY(A,E)) AS proc_cnt',
                            'FIRST(tstamp OF ANY(E)) AS starttime'])

npath_sessions1.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(5)

In [ ]:
knee_path_results1 = npath_sessions1.result

In [ ]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
res = knee_path_results1\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(10)\
                    .to_pandas()
sankeyPlot(res,"Path to Knee ARTHROSCOPY")

<p style = 'font-size:16px;font-family:Arial'>The TOP event that led to <b>Knee Arthroscopy</b> is <b>Excis Knee Semilun Cartl</b>.</p>

<p style = 'font-size:16px;font-family:Arial'>To check the details of any path or node we can move the mouse pointer over it and check details. Similarly other nodes and paths can be analyzed using similar functions.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Now let's Check paths that led to Other Diagnostic Physical Therapy Procedure.</b></p>
<p style = 'font-size:16px;font-family:Arial'>As seen in the above analysis ,Knee replacement was suggested after these procedures also.</p>

In [ ]:
npath_sessions_oth = NPath(data1 = knee_claims_sessions_out.result, 
                      data1_partition_column = ['memberid'], 
                      data1_order_column = ['tstamp','proceduredesc'], 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['proceduredesc in (\'OTHER DIAGNOSTIC PHYSICAL THERAPY PROCEDURE\') as E', 'proceduredesc not in (\'OTHER DIAGNOSTIC PHYSICAL THERAPY PROCEDURE\') as A'], 
                      pattern = 'A{1,5}*.E', 
                      result = ['ACCUMULATE(proceduredesc OF ANY(A,E)) AS path',
                            'FIRST(diagcode OF ANY(A,E)) AS diagnosis', 
                            'FIRST(memberid OF ANY(A,E)) AS unique_memberid',   
                            'COUNT(proceduredesc OF ANY(A,E)) AS proc_cnt',
                            'FIRST(tstamp OF ANY(E)) AS starttime'])

npath_sessions_oth.result\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(5)

In [ ]:
knee_path_results_oth = npath_sessions_oth.result

In [ ]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
res = knee_path_results_oth\
                    .groupby(['path'])\
                    .count()\
                    .sort('count_unique_memberid',ascending=False)\
                    .head(10)\
                    .to_pandas()
sankeyPlot(res,"Path to Other Diagostic Procedures")

<p style = 'font-size:16px;font-family:Arial'>As seen in the above sankey diagram there are various events that led to Other Diagnostic Procedures. The TOP event being<b> Oth Arthrotomy-Knee</b> followed by <b>Knee Arthroscopy.</b></p>

<p style = 'font-size:16px;font-family:Arial'>To check the details of any path or node we can move the mouse pointer over it and check details. Thus, we have seen that using the nPath function we can check on various Events that led to different procedures and finally Total Knee Replacement.</p>

<p style = 'font-size:20px;font-family:Arial'><b>Total Cost for Knee Replacements.</b></p>
<p style = 'font-size:16px;font-family:Arial'>Let's check how we can use the nPath function to find the average cost involved in different paths that lead to Total Knee Replacement.</p>


In [ ]:
npath_sessions_cost = NPath(data1 = knee_claims_sessions_out.result, 
                      data1_partition_column = ['memberid'], 
                      data1_order_column = ['tstamp','proceduredesc'], 
                      mode = 'NONOVERLAPPING', 
                      symbols = ['proceduredesc in (\'TOTAL KNEE REPLACEMENT\') as E', 'proceduredesc not in (\'TOTAL KNEE REPLACEMENT\') as A'], 
                      pattern = 'A{1,5}*.E', 
                      result = ['ACCUMULATE(proceduredesc OF ANY(A,E)) AS path',
                            'sum(amount OF ANY(A,E)) AS total_cost', 
                            'FIRST(memberid OF ANY(A,E)) AS unique_memberid',   
                            'COUNT(proceduredesc OF ANY(A,E)) AS proc_cnt',
                            'FIRST(tstamp OF ANY(E)) AS starttime'])

In [ ]:
knee_path_cost_results=npath_sessions_cost.result
# knee_path_cost_results

In [ ]:
df=knee_path_cost_results.select(['path','total_cost']).groupby('path').mean().to_pandas()


In [ ]:
df_graph=df.sort_values(['mean_total_cost'],ascending=False).groupby('path').head(10)
df1=df_graph.head(10)

In [ ]:
import matplotlib.pyplot as mp
df1.plot(x="path", y="mean_total_cost", kind="bar",figsize=(20, 9))

# displaying bar graph
mp.xticks(rotation='vertical')
mp.rcParams['xtick.labelsize'] = 20 
mp.legend()
mp.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Top 10 paths based on the Total Cost required for Total Knee Replacement.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>Thus, using the unique analytic capabilities of Vantage for understanding customer and user behavior over time. In fact, these analytic techniques can be applied at massive scale to derive more accurate results. Then, these results can be combined with other analytics to create more advanced and accurate prediction models. Vantage also allows organizations to scale these models horizontally, by training segmented models per region, user type, etc., or vertically, by combining data from millions or billions of interactions. All of this can be deployed operationally to understand and predict actions in real-time.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Cleanup</b></p>


<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_KneeReplacement');" 
#Takes 45 seconds

In [ ]:
# Removing the connection and clearing session
remove_context()

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>Resources</b>
<p style = 'font-size:16px;font-family:Arial'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial'><b>Links</b>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/reader/eteIDCTX4O4IMvazRMypxQ/uDjppX7PJInABCckgu~KFg'>Teradata Python Package User Guide </a> </li>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/reader/GsM0pYRZl5Plqjdf9ixmdA/MzdO1q_t80M47qY5lyImOA'>Teradataml Python Reference </a></li>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/reader/CWVY0AJy8wyyf7Sm0EsK~w/wjkE42ypEfeMkRFOIqVXfQ'>Teradata nPath Function Reference:</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://docs.teradata.com/reader/CWVY0AJy8wyyf7Sm0EsK~w/RNbOiUg9~r~cxSZHrR~sFQ'>Teradata Sessionize Function Reference</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://pandas.pydata.org/docs/user_guide/index.html'>Python Pandas Reference</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href = 'https://plotly.com/'>Plotly Reference</a></li>
</p>    

<p style = 'font-size:18px;font-family:Arial'><b>Filters:</b></p>
<li style = 'font-size:16px;font-family:Arial'><b>Industry:</b> HealthCare </li>
<li style = 'font-size:16px;font-family:Arial'><b>Functionality:</b> Path Analytics</li>
<li style = 'font-size:16px;font-family:Arial'><b>Use Case:</b> Total Knee Replacement</li></p>
<p style = 'font-size:18px;font-family:Arial'><b>Related Resources:</b></p>
<li style = 'font-size:16px;font-family:Arial'><a href='https://teradata.seismic.com/Link/Content/DCGBP9J9gjD288TPcG3HFgXDHDW8'>Broken Digital Journeys CX Solution Accelerator Demo via Python Video - External - SP004183</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href='https://www.teradata.com/Blogs/Forecasting-COVID-19-Using-Teradata-Vantage'>Forecasting COVID-19 Using Teradata Vantage</a></li>
<li style = 'font-size:16px;font-family:Arial'><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li></p>



<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>